In [1]:
%pip install git+https://github.com/neelnanda-io/TransformerLens.git
%pip install plotly
# %pip install sentencepiece # for Llama-2-7b-hf only
%pip install accelerate
%pip install -U kaleido
%pip install nbformat
%pip install seaborn
%pip install fancy_einsum

  Cloning https://github.com/neelnanda-io/TransformerLens.git to /tmp/pip-req-build-82_l4cq1
  Running command git clone --filter=blob:none --quiet https://github.com/neelnanda-io/TransformerLens.git /tmp/pip-req-build-82_l4cq1
  Resolved https://github.com/neelnanda-io/TransformerLens.git to commit 3f929b1d142b8f82bfbb8ae30e69bab7f76cadf3
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
from transformer_lens import HookedTransformer

In [3]:
# model loading
model = HookedTransformer.from_pretrained(
    "pythia-70m",
    center_unembed=True,
    center_writing_weights=True,
    fold_ln=True,
)

Using pad_token, but it is not set yet.


Loaded pretrained model pythia-70m into HookedTransformer


In [4]:
#Bunch of import staements
import wandb
import IPython
from IPython.display import Image, display
import torch
import gc
from tqdm import tqdm
import networkx as nx
import os
import torch
import huggingface_hub
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import einops
from tqdm import tqdm
import yaml
from transformers import AutoModelForCausalLM, AutoConfig, AutoTokenizer

import matplotlib.pyplot as plt
import plotly.express as px
import plotly.io as pio
from plotly.subplots import make_subplots
import plotly.graph_objects as go

from transformer_lens.hook_points import HookedRootModule, HookPoint
from transformer_lens.HookedTransformer import (
    HookedTransformer,
)
try:
    from acdc.tracr_task.utils import (
        get_all_tracr_things,
        get_tracr_model_input_and_tl_model,
    )
except Exception as e:
    print(f"Could not import `tracr` because {e}; the rest of the file should work but you cannot use the tracr tasks")
from acdc.docstring.utils import get_all_docstring_things
from acdc.acdc_utils import (
    make_nd_dict,
    reset_network,
    shuffle_tensor,
    cleanup,
    ct,
    TorchIndex,
    Edge,
    EdgeType,
)  # these introduce several important classes !!!

from acdc.TLACDCCorrespondence import TLACDCCorrespondence
from acdc.TLACDCInterpNode import TLACDCInterpNode
from acdc.TLACDCExperiment import TLACDCExperiment

from acdc.acdc_utils import (
    kl_divergence,
)
from acdc.ioi.utils import (
    get_all_ioi_things,
    get_gpt2_small,
)
from acdc.induction.utils import (
    get_all_induction_things,
    get_validation_data,
    get_good_induction_candidates,
    get_mask_repeat_candidates,
)
from acdc.greaterthan.utils import get_all_greaterthan_things
from acdc.acdc_graphics import (
    build_colorscheme,
    show,
)
import argparse

torch.autograd.set_grad_enabled(False)

In [5]:
#set all the arguments for the ACDC  Experiment
ipython = get_ipython()
parser = argparse.ArgumentParser(description="Used to launch ACDC runs. Only task and threshold are required")

task_choices = ['ioi', 'docstring', 'induction', 'tracr-reverse', 'tracr-proportion', 'greaterthan']
parser.add_argument('--task', type=str, required=True, choices=task_choices, help=f'Choose a task from the available options: {task_choices}')
parser.add_argument('--threshold', type=float, required=True, help='Value for THRESHOLD')
parser.add_argument('--first-cache-cpu', type=str, required=False, default="True", help='Value for FIRST_CACHE_CPU (the old name for the `online_cache`)')
parser.add_argument('--second-cache-cpu', type=str, required=False, default="True", help='Value for SECOND_CACHE_CPU (the old name for the `corrupted_cache`)')
parser.add_argument('--zero-ablation', action='store_true', help='Use zero ablation')
parser.add_argument('--using-wandb', action='store_true', help='Use wandb')
parser.add_argument('--wandb-entity-name', type=str, required=False, default="remix_school-of-rock", help='Value for WANDB_ENTITY_NAME')
parser.add_argument('--wandb-group-name', type=str, required=False, default="default", help='Value for WANDB_GROUP_NAME')
parser.add_argument('--wandb-project-name', type=str, required=False, default="acdc", help='Value for WANDB_PROJECT_NAME')
parser.add_argument('--wandb-run-name', type=str, required=False, default=None, help='Value for WANDB_RUN_NAME')
parser.add_argument("--wandb-dir", type=str, default="/tmp/wandb")
parser.add_argument("--wandb-mode", type=str, default="online")
parser.add_argument('--indices-mode', type=str, default="normal")
parser.add_argument('--names-mode', type=str, default="normal")
parser.add_argument('--device', type=str, default="cuda")
parser.add_argument('--reset-network', type=int, default=0, help="Whether to reset the network we're operating on before running interp on it")
parser.add_argument('--metric', type=str, default="kl_div", help="Which metric to use for the experiment")
parser.add_argument('--torch-num-threads', type=int, default=0, help="How many threads to use for torch (0=all)")
parser.add_argument('--seed', type=int, default=1234)
parser.add_argument("--max-num-epochs",type=int, default=100_000)
parser.add_argument('--single-step', action='store_true', help='Use single step, mostly for testing')
parser.add_argument("--abs-value-threshold", action='store_true', help='Use the absolute value of the result to check threshold')

if ipython is not None:
    # We are in a notebook
    # you can put the command you would like to run as the ... in r"""..."""
    args = parser.parse_args(
        [line.strip() for line in r"""--task=induction\
--zero-ablation\
--threshold=0.71\
--indices-mode=reverse\
--first-cache-cpu=False\
--second-cache-cpu=False\
--max-num-epochs=100000""".split("\\\n")]
    )
else:
    # read from command line
    args = parser.parse_args()

# Process args

if args.torch_num_threads > 0:
    torch.set_num_threads(args.torch_num_threads)
torch.manual_seed(args.seed)

TASK = args.task
if args.first_cache_cpu is None: # manage default
    ONLINE_CACHE_CPU = True
elif args.first_cache_cpu.lower() == "false":
    ONLINE_CACHE_CPU = False
elif args.first_cache_cpu.lower() == "true":
    ONLINE_CACHE_CPU = True
else: 
    raise ValueError(f"first_cache_cpu must be either True or False, got {args.first_cache_cpu}")
if args.second_cache_cpu is None:
    CORRUPTED_CACHE_CPU = True
elif args.second_cache_cpu.lower() == "false":
    CORRUPTED_CACHE_CPU = False
elif args.second_cache_cpu.lower() == "true":
    CORRUPTED_CACHE_CPU = True
else:
    raise ValueError(f"second_cache_cpu must be either True or False, got {args.second_cache_cpu}")
THRESHOLD = args.threshold  # only used if >= 0.0
ZERO_ABLATION = True if args.zero_ablation else False
USING_WANDB = True if args.using_wandb else False
WANDB_ENTITY_NAME = args.wandb_entity_name
WANDB_PROJECT_NAME = args.wandb_project_name
WANDB_RUN_NAME = args.wandb_run_name
WANDB_GROUP_NAME = args.wandb_group_name
INDICES_MODE = args.indices_mode
NAMES_MODE = args.names_mode
DEVICE = args.device
RESET_NETWORK = args.reset_network
SINGLE_STEP = True if args.single_step else False

In [6]:
tl_model = model # transformerlens model

In [7]:
def validation_metric(x):
    return torch.tensor([1.0]).cuda()

In [8]:
print(validation_metric(23))

tensor([1.], device='cuda:0')


In [9]:
#Let's set some hooks for the model
model.cfg.use_hook_mlp_in = True
model.cfg.use_attn_result = True
model.cfg.use_split_qkv_input = True

In [ ]:
#Here instead of kl-divergence, we want to choose the edge that leads to the maximum gradient path

In [10]:
#our version of the kl divergence
from typing import Optional
def kl_divergence_modified(
    logits,
    base_model_logprobs,
    mask_repeat_candidates: Optional[torch.Tensor] = None,
    last_seq_element_only: bool = True,
    base_model_probs_last_seq_element_only: bool = False,
) -> torch.Tensor:
    # Note: we want base_model_probs_last_seq_element_only to remain False by default, because when the Docstring
    # circuit uses this, it already takes the last position before passing it in.

    if last_seq_element_only:
        logits = logits[:, -1, :]

    if base_model_probs_last_seq_element_only:
        base_model_logprobs = base_model_logprobs[:, -1, :]

    logprobs = F.log_softmax(logits, dim=-1)
    kl_div = F.kl_div(logprobs, base_model_logprobs, log_target=True, reduction="none").sum(dim=-1)

    if mask_repeat_candidates is not None:
        assert kl_div.shape == mask_repeat_candidates.shape, (kl_div.shape, mask_repeat_candidates.shape)
        answer = kl_div[mask_repeat_candidates]
    elif not last_seq_element_only:
        assert kl_div.ndim == 2, kl_div.shape
        answer = kl_div.view(-1)
    else:
        answer = kl_div

    return answer.mean

In [11]:
tl_model.reset_hooks()
second_metric = None

# Save some mem
gc.collect()
torch.cuda.empty_cache()
toks_int_values = model.to_tokens("this is clean data")
toks_int_values_other = model.to_tokens("this is corrupt data")
logits, _ = model.run_with_cache(toks_int_values)

In [12]:
import functools
partial_kl_divergence = functools.partial(
    kl_divergence,
    base_model_logprobs=logits,
    last_seq_element_only=False,
    return_one_element=True
)

In [13]:
#set up another task using kl_divergence instead
tl_model.reset_hooks()
second_metric = None

# Save some mem
gc.collect()
torch.cuda.empty_cache()

# Setup wandb if needed
if WANDB_RUN_NAME is None or IPython.get_ipython() is not None:
    WANDB_RUN_NAME = f"{ct()}{'_randomindices' if INDICES_MODE=='random' else ''}_{THRESHOLD}{'_zero' if ZERO_ABLATION else ''}"
else:
    assert WANDB_RUN_NAME is not None, "I want named runs, always"

tl_model.reset_hooks()
exp = TLACDCExperiment(
    model=tl_model,
    threshold=THRESHOLD,
    using_wandb=USING_WANDB,
    wandb_entity_name=WANDB_ENTITY_NAME,
    wandb_project_name=WANDB_PROJECT_NAME,
    wandb_run_name=WANDB_RUN_NAME,
    wandb_group_name=WANDB_GROUP_NAME,
    wandb_dir=args.wandb_dir,
    wandb_mode=args.wandb_mode,
    wandb_config=args,
    zero_ablation=ZERO_ABLATION,
    abs_value_threshold=args.abs_value_threshold,
    ds=toks_int_values,
    ref_ds=toks_int_values_other,
    metric=partial_kl_divergence,
    second_metric=None,
    verbose=True,
    indices_mode=INDICES_MODE,
    names_mode=NAMES_MODE,
    corrupted_cache_cpu=CORRUPTED_CACHE_CPU,
    hook_verbose=False,
    online_cache_cpu=ONLINE_CACHE_CPU,
    add_sender_hooks=True,
    add_receiver_hooks=False,
    remove_redundant=False,
)

/opt/conda/lib/python3.10/site-packages/acdc/TLACDCExperiment.py:132: UserWarning: We shall overwrite the ref_ds with zeros.
  warnings.warn("We shall overwrite the ref_ds with zeros.")


dict_keys(['blocks.5.hook_resid_post', 'blocks.5.hook_mlp_out', 'blocks.5.hook_mlp_in', 'blocks.5.attn.hook_result', 'blocks.5.attn.hook_q', 'blocks.5.hook_q_input', 'blocks.5.attn.hook_k', 'blocks.5.hook_k_input', 'blocks.5.attn.hook_v', 'blocks.5.hook_v_input', 'blocks.4.hook_mlp_out', 'blocks.4.hook_mlp_in', 'blocks.4.attn.hook_result', 'blocks.4.attn.hook_q', 'blocks.4.hook_q_input', 'blocks.4.attn.hook_k', 'blocks.4.hook_k_input', 'blocks.4.attn.hook_v', 'blocks.4.hook_v_input', 'blocks.3.hook_mlp_out', 'blocks.3.hook_mlp_in', 'blocks.3.attn.hook_result', 'blocks.3.attn.hook_q', 'blocks.3.hook_q_input', 'blocks.3.attn.hook_k', 'blocks.3.hook_k_input', 'blocks.3.attn.hook_v', 'blocks.3.hook_v_input', 'blocks.2.hook_mlp_out', 'blocks.2.hook_mlp_in', 'blocks.2.attn.hook_result', 'blocks.2.attn.hook_q', 'blocks.2.hook_q_input', 'blocks.2.attn.hook_k', 'blocks.2.hook_k_input', 'blocks.2.attn.hook_v', 'blocks.2.hook_v_input', 'blocks.1.hook_mlp_out', 'blocks.1.hook_mlp_in', 'blocks.1.at

In [14]:
import os
ims_directory = 'ims'
if not os.path.exists(ims_directory):
    os.makedirs(ims_directory)
# Now, you can proceed to create the subdirectory 'img_new_1'
for i in range(args.max_num_epochs):
    exp.step(testing=False)
    show(
        exp.corr,
        os.path.join(ims_directory, f"img_new_{i+1}.png"),
        show_full_index=False,
    )
for i in range(args.max_num_epochs):
    exp.step(testing=False)

    show(
        exp.corr,
        f"ims/img_new_{i+1}.png",
        show_full_index=False,
    )

    if IN_COLAB or ipython is not None:
        # so long as we're not running this as a script, show the image!
        display(Image(f"ims/img_new_{i+1}.png"))

    print(i, "-" * 50)
    print(exp.count_no_edges())

    if i == 0:
        exp.save_edges("edges.pkl")

    if exp.current_node is None or SINGLE_STEP:
        break

exp.save_edges("another_final_edges.pkl")

if USING_WANDB:
    edges_fname = f"edges.pth"
    exp.save_edges(edges_fname)
    artifact = wandb.Artifact(edges_fname, type="dataset")
    artifact.add_file(edges_fname)
    wandb.log_artifact(artifact)
    os.remove(edges_fname)
    wandb.finish()

No edge 3772
New metric: 207709323264.0

Node: cur_parent=TLACDCInterpNode(blocks.5.hook_mlp_out, [:]) (self.current_node=TLACDCInterpNode(blocks.5.hook_resid_post, [:]))

Metric after removing connection to blocks.5.hook_mlp_out [:] is 217961381888.0 (and current metric 207709323264.0)
Result is 10252058624.0...so keeping connection
No edge 3772

Node: cur_parent=TLACDCInterpNode(blocks.5.attn.hook_result, [:, :, 0]) (self.current_node=TLACDCInterpNode(blocks.5.hook_resid_post, [:]))

Metric after removing connection to blocks.5.attn.hook_result [:, :, 0] is 207729704960.0 (and current metric 207709323264.0)
Result is 20381696.0...so keeping connection

Node: cur_parent=TLACDCInterpNode(blocks.5.attn.hook_result, [:, :, 1]) (self.current_node=TLACDCInterpNode(blocks.5.hook_resid_post, [:]))

Metric after removing connection to blocks.5.attn.hook_result [:, :, 1] is 207793078272.0 (and current metric 207709323264.0)
Result is 83755008.0...so keeping connection

Node: cur_parent=TLACDCI

ValueError: Program dot not found in path.